### Sentiment Analysis on Womens Clothing
##### Predict sentiments on women's clothing, either as positive or negative


In [1]:
%pip install nltk
%pip install wordcloud

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the neccesary libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import nltk

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import nltk.corpus


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
from matplotlib import rcParams
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk import NaiveBayesClassifier
from nltk.corpus import wordnet 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestClassifier 
from nltk.classify.scikitlearn import SklearnClassifier

In [4]:
# import the data
data = "D:/Data_work/Womens Clothing E-Commerce Reviews.csv"
df = pd.read_csv(data)
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [5]:
# read the number of columns and the index
df.columns
for index, i in enumerate(df.columns):
    print([index],i )

[0] Unnamed: 0
[1] Clothing ID
[2] Age
[3] Title
[4] Review Text
[5] Rating
[6] Recommended IND
[7] Positive Feedback Count
[8] Division Name
[9] Department Name
[10] Class Name


In [6]:
#check for missing values in the data set
df.isnull().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [7]:
df.dropna(inplace = True)

In [8]:
# reset the index after dropping the NA values
df.reset_index(drop=True, inplace =True)
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [9]:
#drop the unncessary values in the dataset
df.drop(["Unnamed: 0","Title"], axis = 1, inplace=True)
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


The useful columns for the Analysis will be Review Text and the Recommended IND to classify the review s negative and postive

In [10]:
#remove spaces in the column titles and replace them with "_"
df.columns = df.columns.str.replace(" ", "_")
df.head()

,Clothing_ID,Age,Review_Text,Rating,Recommended_IND,Positive_Feedback_Count,Division_Name,Department_Name,Class_Name
0,1077,60,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [11]:
# creating review Tuples to store the words with the categories
reviews = []
# going through the recommeneded IND column and get the category
# and index
for (index, category) in enumerate(df.Recommended_IND):
    reviews.append((df.Review_Text[index], category))

In [12]:
reviews[0:4]

[('I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
  0),
 ("I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",
  1),
 ('This shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!',
  1),
 ('I love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress 

#### Cleaning the stopwords

In [13]:
# create the lemmatizer object 
lemmatizer  = WordNetLemmatizer()

In [14]:
# create a list of stopwards
stops = stopwords.words("english")
punctuations = list(string.punctuation)
stops = stops + punctuations
stops, string.punctuation

(['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're",
  "you've",
  "you'll",
  "you'd",
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  "she's",
  'her',
  'hers',
  'herself',
  'it',
  "it's",
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  "that'll",
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'against',
  'between',
  'into',
  'through',
  'during',
  'before',
  'after',
  'above',
  'below',
  'to',
  'from',
  'up',
  'down',
  'in',
  'out',
  'on',
  'off',
  'over',
  'under',
  'again',
  'further',
  'th

In [15]:
# busines stopwords
business_stopwords= ["i'm","would", "look", "ordered", "wear", "fit", "one", "fits","bought", "looks", "also", "got", "think", "even",
                     "tried", "get", "could", "made","way","still", "runs","true" ,"right", "see","online","wearing", "however", "design","purchased","feel","go",
                     "enough","model","though","price","looked","person","better","first","going","try", "body" "bottom","time","many","looking","around","thought",
                     "make","wanted","saw","makes","went","find","found","buy","nan","i've", "since","seems","ok", "girl", "woman"]
stops = stops + business_stopwords
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Lemmatization of the text
Writing a function to lemmatize the words and clean them using the stop word list created

In [16]:
# stop_words after updated with business_stopwords
len(stops)
# parts of speech tagging  to use in the lemmitization
def get_simple_pos(tag):
    if tag.startswith('N') or tag.startswith('J'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
def clean_review(words):
    output_words =[]
    words_tokens = nltk.word_tokenize(words)
    for word in words_tokens:
        if word.lower() not in stops:
            pos = pos_tag([word]) # get the part of speech
            
            clean_word = (lemmatizer.lemmatize(word.lower(),
                                              pos = get_simple_pos(tag)) for word, tag in pos)
            output_words.append(', '.join(map(str,clean_word)))
    return output_words
            

In [18]:
# test the function

clean_review("My cats are running away from my arms")

['cat', 'run', 'away', 'arm']

In [19]:
cleaned_review = [(clean_review(text), category) for text, category in reviews
]


In [20]:
len(cleaned_review)

19662

In [22]:
#looking at the first 5 reviews
cleaned_review[0:6]

[(['high',
   'hope',
   'dress',
   'really',
   'work',
   'initially',
   'petite',
   'small',
   'usual',
   'size',
   'outrageously',
   'small',
   'small',
   'fact',
   'zip',
   'reorder',
   'petite',
   'medium',
   'ok.',
   'overall',
   'top',
   'half',
   'comfortable',
   'nicely',
   'bottom',
   'half',
   'tight',
   'layer',
   'several',
   'somewhat',
   'cheap',
   'net',
   'layer',
   'imo',
   'major',
   'flaw',
   'net',
   'layer',
   'sewn',
   'directly',
   'zipper',
   'c'],
  0),
 (['love',
   'love',
   'love',
   'jumpsuit',
   "'s",
   'fun',
   'flirty',
   'fabulous',
   'every',
   'nothing',
   'great',
   'compliment'],
  1),
 (['shirt',
   'flatter',
   'due',
   'adjustable',
   'front',
   'tie',
   'perfect',
   'length',
   'legging',
   'sleeveless',
   'pair',
   'well',
   'cardigan',
   'love',
   'shirt'],
  1),
 (['love',
   'tracy',
   'reese',
   'dress',
   'petite',
   '5',
   'foot',
   'tall',
   'usually',
   '0p',
   'bran

### Splitting the data
* 75% for training
* 25% for testing

In [30]:
print("Trainig data = {}".format(round(0.75*len(cleaned_review))))
print("Testing data = {}".format(round(0.25*len(cleaned_review))))

training_data = cleaned_review[0:14746]
testing_data = cleaned_review[14746: 0]

Trainig data = 14746
Testing data = 4916


In [31]:
# array containing all of the days
word_list = []
for word in training_data:
    word_list+=word[0] # 0 index to get only the words

In [32]:
len(word_list)

392719

In [33]:
# frequency distribution for all words
freq = nltk.FreqDist(word_list)
# most .most_common() method lists the words which occur most frequently
# in the data
common = freq.most_common()
# features are an array of only the top words in word 
#list without The number of words
features = [i[0] for i in common]

In [34]:
print(len(common))
print(len(features))

12805
12805


In [35]:
# most common 5 words
common[0:5]

[('dress', 8091), ('size', 7083), ('love', 6701), ("n't", 5543), ('top', 5450)]

In [37]:
features[0:5]

['dress', 'size', 'love', "n't", 'top']

In [38]:
pwd


'D:\\Projects'